In [ ]:
import os
from plotting.combine_results import find_files
from eval_model import get_model_structure
import numpy as np
from utils.inference_utils import load_names, load_observed
from plotting.plot_utils_bpaitac import histogram
from region_identification_utils import get_trial_metrics, identify_regions, summary_stats
import pandas as pd
from utils.load_model import get_model, get_predictions, load_model
import scipy
import matplotlib.pyplot as plt
import torch


Selected GPU: 0
DEVICE is cuda:0


### Define models

In [2]:
model1_dir = '/data/nchand/analysis/BPcm_250/BP200_L0_0/'
path_model1 = '/data/nchand/analysis/BPcm_250/BP200_L0_0/complete/04-08-2025.15.04/best_model'
model2_dir = '/data/nchand/analysis/BPcm_250/BP203_no_seed_L-1_5/'
path_model2 = '/data/nchand/analysis/BPcm_250/BP203_no_seed_L-1_5/complete/04-06-2025.18.35/best_model'
output_dir = '/data/nchand/analysis/BPcm_250/BP203_200_analysis'
model_type = 'BPcm_250'

n_celltypes = 90
n_filters = 300
seq_len = 998
model_structure1 = get_model_structure(model_type, n_filters, n_celltypes, seq_len=seq_len)
model_structure2 = get_model_structure(model_type, n_filters, n_celltypes, seq_len=seq_len)

model_structure = get_model_structure(model_type, n_filters, n_celltypes, seq_len=seq_len)
print(path_model1 == path_model2)
model1 = load_model(path_model1, model_structure=model_structure1, n_filters=n_filters, verbose=True)
model2 = load_model(path_model2, model_structure=model_structure2, n_filters=n_filters, verbose=False)

199
39
7
199
39
7
199
39
7
False
DEVICE is cuda:0
998 300 90
MODEL STRUCTURE BPcm_250(
  (body): Body(
    (net): Sequential(
      (0): Conv1d(4, 300, kernel_size=(25,), stride=(1,), padding=same)
      (1): DialatedConvs(
        (net): Sequential(
          (0): ResBlock(
            (conv_block): Sequential(
              (0): Conv1d(300, 300, kernel_size=(3,), stride=(1,), padding=same, dilation=(2,))
              (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU()
            )
          )
          (1): ResBlock(
            (conv_block): Sequential(
              (0): Conv1d(300, 300, kernel_size=(3,), stride=(1,), padding=same, dilation=(4,))
              (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU()
            )
          )
          (2): ResBlock(
            (conv_block): Sequential(
              (0): Conv1d(300, 300, kernel_size=(3,), stride=(1,

/homes/gws/nchand/MostafaviLab/bpAITAC/load_model.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(saved_model_path))


DEVICE is cuda:0
998 300 90


### Load data

In [25]:
cell_names = np.load("/data/nchand/ImmGen/mouse/BPprofiles1000/ImmGenATAC1219.peak_matched_in_sorted.sl10004sh-4.celltypes.npy")

b6_seq_files = np.load('/data/mostafavilab/ImgenATAC/F1_chr11_chr16_seq/b6seqs_padded_chr11chr16.npz')
print(b6_seq_files['names'].shape)
print(b6_seq_files['seqs'].shape)
b6_names = b6_seq_files['names']
b6_seqs = b6_seq_files['seqs']

castseqs_files = np.load('/data/mostafavilab/ImgenATAC/F1_chr11_chr16_seq/castseqs_padded_chr11chr16.npz')
castseqs_names = castseqs_files['names']
castseqs_seqs = castseqs_files['seqs']


(25207,)
(25207, 4, 998)


### Make predictions

In [26]:
DEVICE = 'cuda'
from tqdm import tqdm
import torch

def pred(model, onehot_seq, n_celltypes=90, batch_size=100):
    torch.cuda.empty_cache()
    model.to(DEVICE)
    counts = torch.zeros((0, n_celltypes)).to(DEVICE)
    onehot_seq = onehot_seq.astype(np.float32)
    seq = torch.from_numpy(onehot_seq)

    # Calculate number of batches (ceiling division)
    n_batches = (len(seq) + batch_size - 1) // batch_size
    
    # Process all batches including the final partial batch
    for i in tqdm(range(n_batches)):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(seq))
        X = seq[start_idx:end_idx]
        bias = torch.zeros((X.shape[0], X.shape[2]))
        with torch.no_grad():
            profile, scalar = model(X.to(DEVICE), bias.to(DEVICE))
            counts = torch.cat((counts, scalar), 0)
    
    return counts


In [27]:
b6_counts_model1 = pred(model1, b6_seqs)
print(b6_seqs.shape)
print(b6_counts_model1.shape)

b6_counts_model2 = pred(model2, b6_seqs)
print(b6_seqs.shape)
print(b6_counts_model2.shape)


100%|████████████████████████████████████████████████████████████████| 253/253 [00:15<00:00, 16.74it/s]


(25207, 4, 998)
torch.Size([25207, 90])


100%|████████████████████████████████████████████████████████████████| 253/253 [00:14<00:00, 16.89it/s]

(25207, 4, 998)
torch.Size([25207, 90])


In [ ]:
from utils.inference_utils import load_data
train_loader, val_loader, test_loader = load_data('/data/nchand/ImmGen/mouse/BPprofiles1000/memmaped/complete_shallow_deprotinated_bias_quantile_normalized_4.1.25/memmap/info.txt', batch_size=20)
eval_set='validation'
get_predictions(model2, train_loader=train_loader, val_loader=val_loader, test_loader=test_loader, n_celltypes=90, get_scalar_prediction=False, 
                get_scalar_observed=False, get_profile_observed=True, get_profile_prediction=True, eval_set=eval_set, bin_size=1, model_dir=model2_dir, seq_len=998, seq_out_len=250)


USING validation data loader
batch num 0
bias shape before it goes into model torch.Size([20, 998])
jsd mean tensor(0.7222)
batch num 1
bias shape before it goes into model torch.Size([20, 998])
jsd mean tensor(0.7402)
batch num 2
bias shape before it goes into model torch.Size([20, 998])
jsd mean tensor(0.7455)
batch num 3
bias shape before it goes into model torch.Size([20, 998])
jsd mean tensor(0.7498)
batch num 4
bias shape before it goes into model torch.Size([20, 998])
jsd mean tensor(0.7365)
batch num 5
bias shape before it goes into model torch.Size([20, 998])
jsd mean tensor(0.7263)
batch num 6
bias shape before it goes into model torch.Size([20, 998])
jsd mean tensor(0.7255)
batch num 7
bias shape before it goes into model torch.Size([20, 998])
jsd mean tensor(0.7307)
batch num 8
bias shape before it goes into model torch.Size([20, 998])
jsd mean tensor(0.7261)
batch num 9
bias shape before it goes into model torch.Size([20, 998])
jsd mean tensor(0.7267)
batch num 10
bias sha

KeyboardInterrupt: 

In [29]:
print(np.array_equal(b6_counts_model1.cpu(), b6_counts_model2.cpu()))

False


In [30]:
bp6_save_path = '/data/mostafavilab/ForAlex/F1_experiment/b6seqs_padded_chr11chr16_predictions_qnorm_BP200_203.npz'
np.savez(bp6_save_path, seqs=b6_seqs, names=b6_names, cellnames=cell_names, lambda0_pred=b6_counts_model1.cpu(), lambdap5_pred=b6_counts_model2.cpu())

In [31]:

castseqs_counts_model1 = pred(model1, castseqs_seqs)
print(castseqs_seqs.shape)
print(castseqs_counts_model1.shape)

castseqs_counts_model2 = pred(model2, castseqs_seqs)
print(castseqs_seqs.shape)
print(castseqs_counts_model2.shape)

  0%|                                                                          | 0/253 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████| 253/253 [00:15<00:00, 16.74it/s]


(25207, 4, 998)
torch.Size([25207, 90])


100%|████████████████████████████████████████████████████████████████| 253/253 [00:15<00:00, 16.63it/s]

(25207, 4, 998)
torch.Size([25207, 90])


In [32]:
print(np.array_equal(castseqs_counts_model1.cpu(),castseqs_counts_model2.cpu()))

False


In [33]:
castseqs_save_path = '/data/mostafavilab/ForAlex/F1_experiment/castseqs_padded_chr11chr16_predictions_qnorm_BP200_203.npz'
np.savez(castseqs_save_path, seqs=castseqs_seqs, names=castseqs_names, cellnames=cell_names, lambda0_pred=castseqs_counts_model1.cpu(), lambdap5_pred=castseqs_counts_model2.cpu())

In [35]:
np.load('/data/mostafavilab/ForAlex/F1_experiment/castseqs_padded_chr11chr16_predictions_qnorm_BP200_203.npz').files

['seqs', 'names', 'cellnames', 'lambda0_pred', 'lambdap5_pred']

In [34]:
test =np.load(castseqs_save_path)
model1_p = test['lambda0_pred']
model2_p = test['lambdap5_pred']

print(np.array_equal(model1_p,model2_p))

False


## Testing outputs of profile prediction

In [ ]:
import os
from plotting.combine_results import find_files
from eval_model import get_model_structure
import numpy as np
from utils.inference_utils import load_names, load_observed
from plotting.plot_utils_bpaitac import histogram
from utils.region_identification_utils import get_trial_metrics, identify_regions, summary_stats
import pandas as pd
from l, get_predictions, load_model
import scipy
import matplotlib.pyplot as plt
from plotting.plot_results import plot_profile
import torch


In [16]:
n_celltypes = 90
n_filters = 300
seq_len = 998
model_type = 'BPcm_250'

model_name = 'BP200_L-1_5'
model_dir_3 = '/homes/gws/nchand/MostafaviLab/results/BPcm_250/BP200_sample_L-1_5/complete/03-31-2025.16.07/best_loss_model.pth'
model_structure3 = get_model_structure(model_type, n_filters, n_celltypes, seq_len=seq_len)
model = load_model(model_dir_3, model_structure=model_structure3, n_filters=n_filters, verbose=False)





199
39
7
DEVICE is cuda:3
998 300 90


/homes/gws/nchand/MostafaviLab/bpAITAC/load_model.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(saved_model_path))


# grab some test data

In [17]:
info_file = '/data/nchand/ImmGen/mouse/BPprofiles1000/memmaped/complete_bias_corrected_normalized_3.7.23/memmap/info.txt'
onehot_seqs = load_observed(info_file, dataset_type='test', data_name='onehot')
bp_counts = load_observed(info_file, dataset_type='test', data_name='bp_counts')
total_counts = load_observed(info_file, dataset_type='test', data_name='total_counts')
bias = load_observed(info_file, dataset_type='test', data_name='bias')


file name /data/nchand/ImmGen/mouse/BPprofiles1000/memmaped/complete_bias_corrected_normalized_3.7.23/memmap/test.onehot.dat
file name /data/nchand/ImmGen/mouse/BPprofiles1000/memmaped/complete_bias_corrected_normalized_3.7.23/memmap/test.bp_counts.dat
file name /data/nchand/ImmGen/mouse/BPprofiles1000/memmaped/complete_bias_corrected_normalized_3.7.23/memmap/test.total_counts.dat
file name /data/nchand/ImmGen/mouse/BPprofiles1000/memmaped/complete_bias_corrected_normalized_3.7.23/memmap/test.bias.dat


## Find very open region and celltype

In [18]:
possible_idx = np.argwhere(np.max(total_counts,axis=1)>500)[:, 0]
print(possible_idx.shape)

idx = possible_idx[30]

(3977,)


## compare pred to actual for bp counts

In [19]:
def pred_single_seq(model, onehot_seqs, bias, seq_idx):
    idx = seq_idx
    X = onehot_seqs[idx:idx+1]
    X = X.transpose(0, 2, 1)
    print(X.shape)
    bias = bias[idx:idx+1, 0, :]
    bias = torch.from_numpy(bias)
    print('bias shape', bias.shape)
    # bias = torch.zeros((X.shape[0], X.shape[2]))

    torch.cuda.empty_cache()
    model.to(DEVICE)
    X = X.astype(np.float32)
    X = torch.from_numpy(X)

    with torch.no_grad():
        profile, scalar = model(X.to(DEVICE), bias.to(DEVICE))
    
    return profile.cpu(), scalar.cpu()


In [20]:
profile, scalar = pred_single_seq(model, onehot_seqs=onehot_seqs, bias=bias, seq_idx=idx)
pred()
profile = profile.numpy()

(1, 4, 998)
bias shape torch.Size([1, 998])


In [21]:
scalar

tensor([[17.6067, 21.8778, 11.5930, 10.0835, 12.3181, 14.4984, 11.8603,  8.4140,
         16.0912, 10.0619, 11.8849, 11.5739, 14.2535, 10.3016,  8.9254, 13.0301,
         10.7061, 11.7486, 15.0419, 14.9377, 13.2589, 12.0105, 13.4672, 11.5037,
         10.4869, 33.0868, 16.7306, 13.0907,  9.3825,  7.1716,  9.6076,  7.9272,
          6.4446,  0.0000, 26.1345,  9.2177, 12.5881, 28.0891, 15.1897, 24.4267,
          8.6167, 13.5006,  9.4757,  7.7642,  9.2850, 23.0289, 20.1094,  7.9458,
          8.7230,  7.9583, 10.3213,  9.8266,  9.7434, 13.2195, 14.2532, 13.7673,
          0.0000,  7.2588, 11.2509, 14.3758, 16.7616, 13.2703, 18.1315, 13.8629,
         16.5219, 33.4187, 12.2921, 12.7030, 10.2662, 11.6177, 13.9978, 10.3168,
         12.9551, 12.8454, 15.4602, 12.8335, 14.8131, 11.0897, 13.9049,  7.9625,
         13.5324,  7.9548,  8.1671, 20.3369, 16.9789, 15.0423, 29.1560, 19.9712,
         21.4915, 13.4858]])

In [22]:
# find most open celltype:
cell_idx = np.argmax(total_counts[idx])
print('cell idx', cell_idx)

bp_counts_center = bp_counts[idx, cell_idx, 375:625]
bp_distribution = bp_counts_center / np.sum(bp_counts_center)
print(bp_distribution.shape)
print('profile shape', profile[0, cell_idx].shape)

title = f'test_{model_name}_idx{idx}'
outpath = title + '.png'
plot_profile(bp_distribution, profile[0, cell_idx], title=title, out_path=outpath)
print('saved at ', outpath)

cell idx 25
(250,)
profile shape (250,)
saved at  test_BP200_L-1_5_idx249.png
